# 1. Types

All values in Julia have a specific Type. The Type system (i.e. the hierarchy of "data identities" anything in Julia has) is the the basis of multiple dispatch (next section). 
Julia's type system is dynamic but also allows to indicate that certain values are of specific type (i.e. static typing).

**Quick important notice: "Types" are written in CamelCase by convention: `String`, `AbstractVector`, `Float64` while "values" and "functions" use lowercase `string`, `vector`, `float`.**

## Type tree

All values in Julia have a type that belongs to a single, fully connected type tree, all nodes of which are equally first-class as types. There is no distinction between native Julia types and user-created ones.

The "central" node of this tree is `Any`, which literally means any possible value. The functions `subtypes` and `supertype` show the children or the parent of a node in this tree.

In [1]:
subtypes(Float64) # this does not have subtypes, it is a "leaf" of the Type tree

Type[]

In [2]:
supertype(Float64)

AbstractFloat

Notice that `subtypes/supertype` must take a Type as an input, not a value! E.g. `subtypes(5.0)` fails, as `5.0` is not a type, but it is instead an *instance* of the type `Float64`.

In [3]:
supertype(AbstractFloat)

Real

In [4]:
supertype(Real)

Number

In [5]:
supertype(Number)

Any

We've reached the "central" node of the Type tree. Here is a nice convenient way to show the rest of the tree that starts from a given Type:

In [5]:
using AbstractTrees; AbstractTrees.children(x::Type) = subtypes(x)

In [6]:
print_tree(Number)

Number
├─ Complex
└─ Real
   ├─ AbstractFloat
   │  ├─ BigFloat
   │  ├─ Float16
   │  ├─ Float32
   │  └─ Float64
   ├─ AbstractIrrational
   │  └─ Irrational
   ├─ Integer
   │  ├─ Bool
   │  ├─ Signed
   │  │  ├─ BigInt
   │  │  ├─ Int128
   │  │  ├─ Int16
   │  │  ├─ Int32
   │  │  ├─ Int64
   │  │  └─ Int8
   │  └─ Unsigned
   │     ├─ UInt128
   │     ├─ UInt16
   │     ├─ UInt32
   │     ├─ UInt64
   │     └─ UInt8
   └─ Rational


In addition, the operator `<:` means "subtype of", for example

In [4]:
Float64 <: Number

true

In [9]:
Float64 <: Int

false

## Abstract and concrete types

Concrete types are anything that can be actually instantiated. Any value that exists in Julia code that is running always has a concrete type. On the other hand, abstract types exist only to establish hierarchical relations between the concrete types. At the moment this sounds abstracted and "out of the blue", but it will become perfectly transparent once we talk about Multiple Dispatch.

In terms of mathematics, the distinction between abstract and concrete is simple: concrete types are leaves of the Type tree, while abstract types are anything else.

Based on the above tree of the `Number` type, one can see that `Number, Real, AbstractFloat` are all abstract types, while `Float64, Float32` are concrete. 

## Custom types

It is of course possible to create your own Types. This happens with the `struct` and `mutable struct` keywords like so

In [7]:
struct A
    x
    y
end

mutable struct B
    x
    y
end

Types are instantiated by calling them as if they are functions, with arguments their fields (this is the "default" constructor, and one can make any arbitrary constructor they want).

In [8]:
a = A(1, 2)

A(1, 2)

In [9]:
b = B(2,3)

B(2, 3)

Here the *values* `a, b` are *instances* of the *Types* `A, B`. The difference between the two is that the `mutable` keyword establishes that any instance of type `B` can be mutated in place. This is not possible for `A`, and the only way to change its values is to make a new instace of `A` alltogether.

Accessing the *fields* of the a Type is done by `a.fieldname`

In [11]:
(a.x, b.x)

(1, 2)

In [12]:
b.x = 5 # possible because b (of type B) is mutable

5

In [13]:
a.x = 5 # impossible

ErrorException: setfield!: immutable struct of type A cannot be changed

Notice that the way we defined `A, B` here is **bad for performance**. The fields `x, y` can have any possible Type, as we added no restrictions to them. This means that once a function gets an instance of `A`, the compiler cannot optimize this function, as it is impossible to know by the type `A` what are the actual data structures involved. This is another form of Type instability that we discussed in the first session!

To avoid this, use the type assertion operator `::` to enforce a Type to the fields of a `struct` definition.

In [14]:
struct Agood
    x::Float64
    y::Float64
    z::String
end

In [15]:
a = Agood(1, 1, "best language evaaaah")

Agood(1.0, 1.0, "best language evaaaah")

This is means that `Agood` is **concretely typed**. (super-duper important by the way)

### Abstract types

As we specified no subtyping rule for `A, B`, they are by definition subtypes of the central node of the tree, `Any`. Abstract types are meaningful for subtyping relations.

Abstract types are defined with the `abstract type` keyword

In [16]:
abstract type C end

At the moment `C` is a subtype of `Any`. It is an abstract type so it is impossible to instantiate:

In [19]:
C(4)

MethodError: MethodError: no constructors have been defined for C

To make `C` useful, we can create some subtypes of `C` using the subtyping operator `<:`

In [17]:
struct D <: C
    d::Number
end

Now `D` is a subtype of `C`:

In [18]:
subtypes(C)

1-element Vector{Any}:
 D

Just to be clear, the above definition of `D` is **also type-unstable**. Sure, it is better to specific that the field of `D` is some subtype of `Number`, but `Number` is still an abstract type!

## Parametric types

Julia types can be parameterized based on other types, meaning that you can indeed make a field of a type to be "anything that is a number", and still be type stable!

This is very useful for reducing code replication as well as leveraging multiple dispatch more. Type-parameterization means that when defining a `struct`, the fields of the struct could be of an aribtrary Type. Type-parameterization uses the curly brakets syntax. For example:

In [19]:
struct Alpha{T}
    a::T
    b::T
end

Here the type `Alpha` can be instantiated using the low level constructor `Alpha{Type}(val1, val2)`, which will convert automatically `val1, val2` to `Type`. For example

In [20]:
a = Alpha{Int64}(1,1)

Alpha{Int64}(1, 1)

The high level constructor `Alpha(val1, val2)` also exist, and will try to deduce what `Type` should be based on the arguments:

In [21]:
Alpha(5, 6)

Alpha{Int64}(5, 6)

In [22]:
Alpha("test", 0.5)

MethodError: MethodError: no method matching Alpha(::String, ::Float64)
Closest candidates are:
  Alpha(::T, !Matched::T) where T at ~/programming/julia/NonlinearDynamicsTextbook/ws2.ipynb:2

Since it is impossible to equivalently convert a string and a float to each other (as `1.0 != "1"`), one cannot instantiate `A` like this.

Here is a second example with two type-parameters:

In [23]:
struct TwoPars{T, F}
    a::T
    b::F
end

In [24]:
TwoPars("string", 0.5)

TwoPars{String, Float64}("string", 0.5)

And a final example:

In [25]:
struct ManyPars{T <: Real, F <: Union{String, Real}}
    a::T
    b::F
    c::Int
end

Here, to instantiate `ManyPars`, `a` must be a subtype of reals, `b` could be a string or a real, while `c` has to be an integer.

# 2. Multiple dispatch
Multiple dispatch is the core programming paradigm of Julia. This notebook presents simple examples that can teach you multiple dispatch. The next session will show you the full power of multiple dispatch, as it is used in the Julia ecosystem.

## What is Multiple Dispatch?

Some handy definitions
* **function**: the name of the "function / process" we are referring to.
* **method**: what actually happens when we call the function with a specific combination of arguments.

*Dispatch* means that when a function call occurs, the language decides somehow which of the function *methods* have to be used. 

### No dispatch
In no dispatch, as in e.g. C, there is nothing to be decided. The method and the function are one and the same. 

### Single dispatch
In single dispatch, as in most object-oriented languages (like Python), it is possible for the same function name to have different methods:
```
array.set_size(args...)
axis.set_size(args...)
```
where `array` could be an instance of something from `numpy` while `axis` could come from `matplotlib`. Here the language dispatches the function `set_size`, depending on the first argument, which is `array` or `axis`. It is important to note that in most object oriented languages, the **method is a property of the type**. 

### Multiple dispatch
Here dispatch occurs based on the type of **every single function argument**, as in
```
set_size(a::Array, args...) = ...
set_size(a::Axis, args...) = ...
set_size(s, a::Array, args...) = ...
set_size(a::Array, b::Vector) = ...
set_size(a::Array, x::Real, y::Real, z::Real) = ...
```
etc. This means that multiple dispatch allows for more exponentially more "expressive power", as one can use all arguments of the function to dispatch on.

### How it works

Multiple dispatch follows easy-to-understand rules based on the Julia type system hierarchy. 

Upon calling a function, Julia will try to find the method that is most specific across all arguments. This means that if a method is defined for both the abstract type combination, as well as the concrete type combination, Julia will always call the more concrete one. This rule also applies to e.g. parametric types, since `Vector{Float64}` is more specialized than `Vector`. This rule also applies to unions, since `Float64` is more specialized than `Union{Float64, String}`.

Two important points:

1. **methods do not belong to the objects!**
2. **new methods can be defined *after* the types have been defined!**

## A simple example

#### Defining some types

In [26]:
abstract type Animal end # this is an abstract type. a supertype of the below

struct Dog <: Animal   # this is a concrete type. a subtype of the above
    name::String
end

struct Cat <: Animal
    name::String
end

Now let's instantiate four animals

In [27]:
fido = Dog("Fido")
rex = Dog("Rex")
whiskers = Cat("Whiskers")
spotty = Cat("Spotty")

Cat("Spotty")

And finally, let's define some functions that take advantage of these `Animal` types, as well as multiple dispatch. Adding a method to a function is done by simply defining the function while also declaring the Types you add a method for:

In [28]:
function encounter(a::Animal, b::Animal)
    verb = meets(a, b)
    println("$(a.name) meets $(b.name) and $verb")
end

meets(a::Animal, b::Animal) = "passes by"

meets (generic function with 1 method)

Both of the above functions are defined on the abstract type level.

In [29]:
encounter(fido, rex)
encounter(fido, spotty)

Fido meets Rex and passes by
Fido meets Spotty and passes by


We now define more specific methods like so:

In [30]:
meets(a::Dog, b::Dog) = "sniffs"
meets(a::Dog, b::Cat) = "chases"
meets(a::Cat, b::Dog) = "hisses"
meets(a::Cat, b::Cat) = "slinks"

meets (generic function with 5 methods)

*(notice how the amount of methods of `meets` increased)*

In [31]:
encounter(fido, rex)
encounter(fido, whiskers)
encounter(whiskers, spotty)
encounter(spotty, rex)

Fido meets Rex and sniffs
Fido meets Whiskers and chases
Whiskers meets Spotty and slinks
Spotty meets Rex and hisses


## Simple extention of multiple dispatch
What if we get a third animal? Like a rabbit? It is easy to extend the system

In [32]:
struct Rabbit <: Animal
    name::String
end

meets(a::Dog, b::Rabbit) = "wiggles its tail"
meets(a::Rabbit, b::Cat) = "hides"

hops = Rabbit("Hops")

Rabbit("Hops")

In [33]:
encounter(rex, hops)
encounter(hops, whiskers)

Rex meets Hops and wiggles its tail
Hops meets Whiskers and hides


In [34]:
encounter(whiskers, hops)

Whiskers meets Hops and passes by


## Inspecting dispatch

To see how many methods a function has to it, and from which module they come, use `methods`:

In [35]:
methods(meets)

# 7 methods for generic function "meets":
[1] meets(a::Dog, b::Dog) in Main at /Users/kern/programming/julia/NonlinearDynamicsTextbook/ws2.ipynb:1
[2] meets(a::Dog, b::Cat) in Main at /Users/kern/programming/julia/NonlinearDynamicsTextbook/ws2.ipynb:2
[3] meets(a::Cat, b::Dog) in Main at /Users/kern/programming/julia/NonlinearDynamicsTextbook/ws2.ipynb:3
[4] meets(a::Cat, b::Cat) in Main at /Users/kern/programming/julia/NonlinearDynamicsTextbook/ws2.ipynb:4
[5] meets(a::Dog, b::Rabbit) in Main at /Users/kern/programming/julia/NonlinearDynamicsTextbook/ws2.ipynb:5
[6] meets(a::Rabbit, b::Cat) in Main at /Users/kern/programming/julia/NonlinearDynamicsTextbook/ws2.ipynb:6
[7] meets(a::Animal, b::Animal) in Main at /Users/kern/programming/julia/NonlinearDynamicsTextbook/ws2.ipynb:6

To see which method is called on a function call signature, use `@which`:

In [36]:
@which meets(whiskers, rex)

meets(a::Cat, b::Dog) in Main at /Users/kern/programming/julia/NonlinearDynamicsTextbook/ws2.ipynb:3

*(notice that in the REPL or the Juno IDE `@which` allows you to go directly to the source code as well)*

In Julia, all "generic" functions are equivalent. The function `+` is in no way superior, or with more priviliges, than the function `meets` we wrote up.

It just has a bit more methods:

In [37]:
+

+ (generic function with 208 methods)

In [38]:
methods(+)

# 208 methods for generic function "+":
[1] +(x::T, y::T) where T<:Union{Int128, Int16, Int32, Int64, Int8, UInt128, UInt16, UInt32, UInt64, UInt8} in Base at int.jl:87
[2] +(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigInt) in Base.GMP at gmp.jl:529
[3] +(c::Union{Int16, Int32, Int64, Int8}, x::BigInt) in Base.GMP at gmp.jl:535
[4] +(c::Union{UInt16, UInt32, UInt64, UInt8}, x::BigFloat) in Base.MPFR at mpfr.jl:390
[5] +(c::Union{Int16, Int32, Int64, Int8}, x::BigFloat) in Base.MPFR at mpfr.jl:398
[6] +(c::Union{Float16, Float32, Float64}, x::BigFloat) in Base.MPFR at mpfr.jl:406
[7] +(x::Union{Dates.CompoundPeriod, Dates.Period}) in Dates at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Dates/src/periods.jl:372
[8] +(Da::LinearAlgebra.Diagonal, Db::LinearAlgebra.Diagonal) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/diagonal.jl:172
[9] +(D::LinearAlgebra.Diagonal, S::LinearAlgebra.Symmetric) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/diagonal.jl:176
[10] +(D::LinearAlgebra.Diagonal{var"#s859", V} where {var"#s859"<:Real, V<:AbstractVector{var"#s859"}}, H::LinearAlgebra.Hermitian) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/diagonal.jl:182
[11] +(x::LinearAlgebra.Diagonal, H::LinearAlgebra.UpperHessenberg) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/hessenberg.jl:107
[12] +(A::LinearAlgebra.Diagonal, B::LinearAlgebra.Bidiagonal) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:125
[13] +(A::LinearAlgebra.Diagonal, B::LinearAlgebra.SymTridiagonal) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:135
[14] +(A::LinearAlgebra.Diagonal, B::LinearAlgebra.Tridiagonal) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:163
[15] +(A::LinearAlgebra.Diagonal{var"#s859", V} where {var"#s859"<:Number, V<:AbstractVector{var"#s859"}}, B::LinearAlgebra.UniformScaling) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:241
[16] +(A::LinearAlgebra.UnitLowerTriangular, B::LinearAlgebra.LowerTriangular) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/triangular.jl:641
[17] +(A::LinearAlgebra.UnitLowerTriangular, B::LinearAlgebra.UnitLowerTriangular) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/triangular.jl:643
[18] +(UL::LinearAlgebra.UnitLowerTriangular, J::LinearAlgebra.UniformScaling) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/uniformscaling.jl:182
[19] +(A::LinearAlgebra.UnitLowerTriangular, B::LinearAlgebra.Bidiagonal) in LinearAlgebra at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/LinearAlgebra/src/special.jl:86
[20] +(r1::LinRange{T}, r2::LinRange{T}) where T in Base at range.jl:1354
[21] +(y::AbstractFloat, x::Bool) in Base at bool.jl:172
[22] +(index1::CartesianIndex{N}, index2::CartesianIndex{N}) where N in Base.IteratorsMD at multidimensional.jl:114
[23] +(r::AbstractRange{<:Dates.TimeType}, x::Dates.Period) in Dates at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Dates/src/ranges.jl:63
[24] +(x::AbstractArray{<:Dates.TimeType}, y::Union{Dates.CompoundPeriod, Dates.Period}) in Dates at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/stdlib/v1.7/Dates/src/deprecated.jl:6
[25] +(A::LinearAlgebra.UpperTriangular, B::LinearAlgebra.UpperTriangul

I won't be going into Modules and namespaces here. But it is important to note that it would be possible for us to extend `Animal` as well as `meets` *even if* they were coming from a different Module (e.g. a different Julia package).

Extention is identical with creating a new method, but one specifies the parent module as well:

In [39]:
Base.:+(a::Animal, b::Animal) = println("$(a.name) and $(b.name) are animals")
Base.:+(a::Cat, b::Cat) = println("$(a.name) and $(b.name) cutie-cats, much cuter than doggos!")

In [40]:
rex + whiskers

Rex and Whiskers are animals


In [41]:
spotty + whiskers

Spotty and Whiskers cutie-cats, much cuter than doggos!


Doing `Base.:+` is only required for infix function symbols like `+`. Other functions do not need `:`
to be specified, i.e. one can do

In [42]:
Base.string(a::Animal) = a.name

In [43]:
string(whiskers)

"Whiskers"

## Dispatch on parametric types


Multiple dispatch is of course respecting parametric types. Recall the definition of `Alpha`:
```julia
struct Alpha{T}
    a::T
    b::T
end
```
For example if we define:

In [44]:
foo(a::Alpha) = "generic function"
foo(a::Alpha{<:Number}) = "specific to A with any number"
foo(a::Alpha{Float64}) = "specific to A with 64-bit floats"

foo (generic function with 3 methods)

then

In [45]:
foo(Alpha("here's","a test"))

"generic function"

In [46]:
foo(Alpha(5, 6))

"specific to A with any number"

In [47]:
foo(Alpha(0.5, 0.6))

"specific to A with 64-bit floats"

To learn more about the type system and multiple dispatch please read the official Julia documentation on [Types](https://pkg.julialang.org/docs/julia/THl1k/1.1.1/manual/types.html) and [Methods](https://pkg.julialang.org/docs/julia/THl1k/1.1.1/manual/methods.html). 

# 3. Generic algorithms

How does one go about writing a generic algorithm through multiple dispatch? Well, it's simple really, just don't specify types unless you absolutely have to!

## Some linear algebra algorithm
Imagine the following simple algorithm, that performs some linear algebra computations involving inner products. `A` is supposed to be a matrix, while `vs` is supposed to be a vector of vectors, and the inner product `v ⋅ A ⋅ v` is performed for every vector.

In [54]:
using LinearAlgebra

function inner_sum(A, vs)
    t = zero(eltype(A))
    for v in vs
        t += inner(v, A, v)
    end
    return t
end 

inner(v, A, w) = dot(v, A*w)

inner (generic function with 1 method)

In [55]:
a = rand(3,3)
vs = [rand(3) for i in 1:5]
inner_sum(a, vs)

6.3712974604091785

* The above code is **generic**: any `A, vs` that support the correct operations (here `*` and `dot` only), will work fine.

* Because Julia compiles all code based on the exact input types, `inner_sum` is also **arbitrarily efficient**. Given standard Julia matrices and vectors it uses BLAS. Given statically-sized arrays it uses super-efficient unrolled operations.

## The "OneHot" vector

In various fields that use linear algebra, like machine learning, a special vector datastracture is used often, called a "OneHot" vector. This vector has a 1 in a single entry, and 0 in all other entries. Having a dedicated structure for this vector is super efficient, because you only need to store 2 numbers to store it in memory, irrespectively of how large it is!

Let's create this in Julia. To make our life easy, we will make this new Type a subtype of `AbstractVector`. This means that all functions that are defined on the abstract level for `AbstractVector` will work with our type as well (provided we implement all the necessary *methods*).

In [50]:
struct OneHotVector <: AbstractVector{Bool}
    len::Int
    ind::Int
end

I "know" (either by reading the documentation or by trial and error) that the necessary methods I need to extend to make `OneHotVector` truly behave like any arbitrary Julia `vector` is (a) a `size` method and (b) a `getindex` method:

In [51]:
# Extend size (so that it behaves like an array)
Base.size(v::OneHotVector) = (v.len, )

# Extend indexing (so that accessing a OneHotVector like v[5] works)
Base.getindex(v::OneHotVector, i::Integer) = i == v.ind

In [52]:
oh = OneHotVector(10, 3) # its already a "true" Julia vector!

10-element OneHotVector:
 0
 0
 1
 0
 0
 0
 0
 0
 0
 0

Does our algorithm already work with such a vector?

In [56]:
vs = [OneHotVector(3, rand(1:3)) for _ in 1:5]

inner_sum(a, vs)

3.2447245758477012

crazy stuff.

## OneHot inner product

It is already great that the above algorithm works with the new `OneHotVector` type. The generic code that one writes with Julia is extensible with **minimal** effort.

*But*, it currently uses the generic fallback `inner(w, A, v) = dot(w, A*v)`. This is inneficient! Let's break this down:

1. `A*v` is a matrix-vector multiplication. It iterates every column of `A` and multiplies them by each entry of `v`.
2. If the vector `v` has 0s everywhere and a 1 somewhere, then this operation in fact means `A[:, v.ind]`. I.e. we select the `v.ind`-th column of the matrix `A`.
3. `dot(w, Av)` is also generic, it indexes into `w` and `Av` and does a pair-wise multiplication.
4. Given the structure of `OneHotVector` we know that `inner(w, A, v)` is in fact just `A[w.ind, v.ind]`!

Increasing the performance of our algorithm is **trivial**. Simply extend `inner` for `OneHotVector`. But first, some benchmarks...

In [57]:
using BenchmarkTools

L = 100
a = rand(L, L)
vs = [OneHotVector(L, rand(1:L)) for _ in 1:5]

# timing before performance extension
@btime inner_sum(a, vs)

  8.764 μs (6 allocations: 4.39 KiB)


1.134395779815352

In [58]:
inner(w::OneHotVector, a, v::OneHotVector) = a[w.ind, v.ind]

inner (generic function with 2 methods)

In [59]:
# timing after performance extension
@btime inner_sum(a, vs)

  12.929 ns (1 allocation: 16 bytes)


1.134395779815352

# 4. Code re-use in Julia

There is an astoning amount of code-reuse and cooperation among different packages of the Julia ecosystem, as we will see in the next session. Naively one would think it is because of the ease of writing generic algorithms. This is in part true, but the reason Julia is the only language that truly succeeds in having all packages play nicely with each other (without each package actively writing glue code) is that you can re-use types from other people!

In the following I'll compare the story of "extending some else's type" in Julia and in Python, and the difference will become apparent.

---

## An implemented type

Let's take as example the type `RGB`, which represents a color, and is defined in the Julia package `ColorTypes`. It is a simple type that bundles a value for red, green and blue, as well as defines some basic operations for the `RGB` type.

## Extending this type
Now someone, for some reason, wants to **extend this existing type** `RGB`. For example, one might want to create a color vector space. 

How is this done in Julia? 
Simple! Extend the functionality of `RGB` by adding new methods that dispatch on `RGB`. 

For example, you could extend **existing functions**
```julia
module ExtentionsToRGB # creates a new module (just for clarity purposes)

using RGB

Base.zero(::RGB) = RGB(0,0,0)
Base.rand(::RGB) = RGB(rand(), rand(), rand())

end
```
etc., or **define new functions**
```julia
# dot product in color space: convert to grayscale and then normalize
dotc(x::RGB, y::RGB) = 0.2*x.r*y.r + 0.771*x.g*y.g + 0.029*x.b*y.b
```

All of this "just works (TM)" in Julia, even if the type `RGB` comes from a completely independent Julia package!

## Is this possible in OOP ?
The above is something completely trivial to do in Julia. How possible it is in object-oriented programming?

In OOP the methods of a "class", are **defined literally textually within the class**. Therefore, if you wanted to extend this `RGB` in Python you would have two options:

1. Edit the original class.
2. Inherit the original class.

### (1) Modifying the original class
There are problems here:
* You have to convice the author of the original package that your code is worthy to be included.
* If everyone convinces the author about their method, the original class becomes huge and the source code becomes an unreadable mess.
* The original developer has to maintain everyone else's whacka-doodle methods.
* You can't change your mind about your extentions without potentially breaking everyone that uses the class `RGB`.

### (2) Inheriting
There are problems here as well:
* If you inherit, you can't "inherit the name"; you have to create a new type `MyRGB` that inherits `RGB`.
* Your extention will **not** actually apply to `RGB`, only to `MyRGB`. Therefore, anyone that creates `RGB` cannot potentially benefit from your code.
* This complexity increases quite fast; someone else makes `YourRGB` object, and then a third person tries to inherit from both `MyRGB` and `YourRGB` and makes a `OurRGB`, etc... Quickly becomes a hellhole!

## Why Julia does better here
Julia's multiple dispatch has none of the problems of (1) and (2). In addition, everyone that does not load the module `ExtentionsToRGB` is completly unaffected by whatever whacka-doodle shady business is happening there. On the other side, with the simple one liner `using ExtentionsToRGB`, you immediatelly gain access to all the extentions that have been done in that package. **And these extensions are valid for the single Julia type `RGB`**. No `MyRGB` nonsense!

The reasons why this is so trivial in Julia is because:
1. You can define new types to which *existing operations* apply
2. You can define new operations which apply to *existing types*

To go deeper into the concepts discussion in this section, you should have a look at this [blog post by Lyndon White](https://white.ucc.asn.au/2020/02/09/whycompositionaljulia.html) or this [talk by Stefan Karpinski](https://www.youtube.com/watch?v=kc9HwsxE1OY).

# 5. Exercises

## Type hierarchies

Create a function `person_info(p)` that takes in any type of a "person" and prints their name and potentially some extra information. For a normal person, only the name is printed. If the person is a student, it should print their name *and* grade. If it is a group leader, then print their name and their group name. If the input to `person_info` is something other than a "person", it should error.

Solve this exercise without using a single `if` statement; only multiple dispatch. These kind of problems seem to be "natural" to solve using `if` statements. However using multiple dispatch instead makes the code clearer, more easy to extend and (typically) more performant, because of how Julia compiles code.

*Hint: given the rules of the exercise, you do not actually need abstract types to solve it. However, it is in general good practice to use them.*

## Indexing of a range type
Create your own `Range` object, which is an efficient iterable container defined by a `(start, step, stop)`, which gives numeric values incremented by `step` starting from `start`, while *ensuring* that the result never exceeds `stop`. You only need to extend `Base.getindex(r::Range, i::Int)` for this exercise. 

Test your range by initializing a range like `r = Range(0, 0.5, 1)`, and checking of `r[1], r[2], r[3]` give you indeed `0.0, 0.5, 1.0` as they should.

This object can efficiently represent this range while storing only these 3 numbers, instead of all elements that theoretically belong to the range.

*Hints:*
1. As a first step, define your `Range` `struct`, preferably by making it a parametric type of 1 type-parameter (think about how many fields it should have).
2. Use the help functionality to learn about the function `getindex`. The Julia syntax `A[i]` is translated to `getindex(A, i)`. 
3. Implement indexing for your Range type via extending `getindex`. Index `1` giving `start`. Be sure it errors for incorrect indices (where the value given by the range would exceed `stop`).

## Iteration of a range type

Continuing from the previous exercise, now make your `Range` object iterable by extending `Base.iterate(r::Range, state = 1)` as well as `Base.length(r::Range)`. It will then be possible to do `collect(range)` or `for a in range ...` or `[cos(r) for r in range]`.

*Hints:*

The [Julia documentation on the iteration API](https://pkg.julialang.org/docs/julia/THl1k/1.1.1/manual/interfaces.html#man-interface-iteration-1) shows that the Julia code block
```julia
for i in iter   # or  "for i = iter"
    # body
end
```

is in fact internally translated into:
```julia
next = iterate(iter)
while next !== nothing
    (i, state) = next
    # body
    next = iterate(iter, state)
end
```

1. To solve this exercise you need to define `Base.iterate(r::Range, state = 1)` which returns either the current element and the next iteration state (typically `i+1`) or `nothing` if there are no elements remaining to be iterated.
2. Test `iterate` for your `Range` by using it in a `for` loop: `for r in myrange; @show r; end`.

* **Bonus point** Did you write generic code? Try to see if your `Range` works with the package `Unitful`. This package provides types that combines numbers with physical units (like Newtons). Load this package with `using Unitful`. Does iterating a `Range(1u"kg", 0.1u"kg", 10u"kg")` work as expected? Make the necessary adjustions so that it does.

## Advanced whackadoodle iterator

Leaveraging the experience of previous exercises, define an iterator object (and of course define `iterate` for it!) that has the following properties (**make an iterator struct, don't create a vector!**):

* It iterates for `n` elements. The first element is always `rand()`.
* At the odd steps of the iteration it returns the previous element divided by `rand()`.
* At the even steps of the iteration it returns the previous element multiplied by `rand()`.

Use this iterator object to efficiently study how its mean value behaves versus `n`, by using the function `mean` (accessible after `using Statistics`). 

*Hint: if you implement this iterator correctly, then you should be able to give it directly to `mean`.*

## Rational numbers

Implement the type `RationalNumber <: Real`, similar to Julia's `Rational`, but without being a parametric type (i.e. both fields are just `Int`). Using the following function
```julia
function _normalize(n::Integer, d::Integer)
    g = gcd(n, d)
    m = d < 0 ? -1 : 1
    (m * n ÷ g, m * d ÷ g)
end
```
define an **inner constructor**: a function `RationalNumber` inside the type-definition, which returns `new(correct arguments...)`. (`new` stands for the type to be created). For example:
```julia
struct A
    a::Float64
    function A(a)
        x = cos(a)
        return new(x) # This creates an instance of A with the value x
    end
end
```
Inner constructors are used to perform specific tests or transformations necessary before instantiating a type. In this exercise the inner constructor simply ensures that if you try to define the rational number 5/10 you in fact create the equivalent (but simpler) 1/2.

Then, implement the operations `+, -, *, /` between `RationalNumber`s, by extenting e.g. `Base.:+(r1::RationalNumber, r2::RationalNumber)`.

Test your code by ensuring that `RationalNumber(5, 10)` indeed has fields `(1, 2)`. Then test all operations with the rationals `1/2` and `1/3`.

* **extra point**: Find out the minimum extention you need to make to your number type so that all trigonometric functions like `cos, tanh`, etc. work out of the box with it. (Hint: it is only one more method definition)